In [1]:
import random
import warnings
import zipfile
from pathlib import Path

import pandas as pd
import tensorflow as tf
import tqdm
warnings.filterwarnings("ignore")

import pyarrow.dataset as ds

2025-10-01 16:51:35.426448: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-01 16:51:35.426482: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-01 16:51:35.427620: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-01 16:51:35.433471: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-01 16:51:36.295290: W tensorflow/compiler/tf2

In [3]:
import pyarrow.dataset as ds
import pyarrow.compute as pc
import pandas as pd

def load_data_efficiently():
    # Caminho para o seu dataset
    dataset_path = "/home/hygo2025/Documents/data/processed_data/enriched_events"

    # Parâmetros para amostragem
    sample_fraction = 0.2
    random_seed = 42

    # 1. Crie o objeto do dataset como antes
    dataset = ds.dataset(
        dataset_path,
        format='parquet',
        partitioning=['dt']
    )

    # 2. Crie a expressão de filtro
    # Isso define a condição `name_raw != 'GalleryClicked'`
    filter_expression = (ds.field('city') == 'Vila Velha') & (ds.field('business_type') == 'RENTAL')


    # 3. Crie um Scanner com o filtro aplicado
    # O Scanner "planeja" a leitura dos dados, mas ainda não os carrega
    scanner = dataset.scanner(filter=filter_expression)

    # 4. Itere sobre os "batches" (pedaços) de dados e faça a amostragem de cada um
    sampled_batches = []
    # O scanner.to_reader() lê os dados filtrados de forma eficiente
    for batch in scanner.to_reader():
        # Converte o pedaço para pandas
        batch_df = batch.to_pandas()
        # Faz a amostragem do pedaço
        sampled_part = batch_df.sample(frac=sample_fraction, random_state=random_seed)
        sampled_batches.append(sampled_part)

    # 5. Concatene todos os pedaços amostrados em um único DataFrame
    if not sampled_batches:
        # Retorna um DataFrame vazio se o filtro não encontrou nada
        return pd.DataFrame()
    else:
        final_data = pd.concat(sampled_batches, ignore_index=True)
        return final_data


In [4]:
data = load_data_efficiently()
print("data shape:", data.shape)

data shape: (118037, 24)


In [5]:
data.head()

,unified_user_id,listing_id,user_id,anonymous_id,name_raw,event_type,collector_timestamp,business_type,event_ts,rating,...,usable_areas,total_areas,bathrooms,bedrooms,suites,parking_spaces,amenities,listing_id_numeric,user_numeric_id,dt
0,94D062F47EDBAAA499546AC41F6297F0FD6045A57ED721...,DBCADAA728D4623E406C676E61465CD35025F1FE394B0A...,None,94D062F47EDBAAA499546AC41F6297F0FD6045A57ED721...,GalleryClicked,OTHER,1706797334226,RENTAL,2024-02-01 14:22:14.226,0.008128,...,107.0,117.0,3,3,1.0,2.0,"['GYM', 'BARBECUE_GRILL', 'KITCHEN', 'ELEVATOR...",1860761,329768,dt=2024-02-01
1,5960DD7B6E63AAA7CA77A30DF9E45C5652F1E4AE17A3B4...,B7DF7E93D4FF82B6B573B0A9ACED39EF9546D8A6789A05...,5960DD7B6E63AAA7CA77A30DF9E45C5652F1E4AE17A3B4...,1D7A1C175FA8FBF963E9915C4D67BB9061365B8023FF14...,GalleryClicked,OTHER,1706799383732,RENTAL,2024-02-01 14:56:23.732,0.008128,...,50.0,50.0,1,1,0.0,0.0,"['AIR_CONDITIONING', 'KITCHEN', 'ELEVATOR', 'I...",1965817,283830,dt=2024-02-01
2,2BCC113546E5AF9B5852D0BFA1738D81A6FB09075743DC...,6772A6F73E7B827F85C35F60E3A759380DE683627F052D...,None,2BCC113546E5AF9B5852D0BFA1738D81A6FB09075743DC...,ListingRendered,VISIT,1706754084854,RENTAL,2024-02-01 02:21:24.854,0.010797,...,260.0,260.0,4,4,2.0,2.0,[],1614965,952233,dt=2024-02-01
3,5960DD7B6E63AAA7CA77A30DF9E45C5652F1E4AE17A3B4...,B7DF7E93D4FF82B6B573B0A9ACED39EF9546D8A6789A05...,5960DD7B6E63AAA7CA77A30DF9E45C5652F1E4AE17A3B4...,1D7A1C175FA8FBF963E9915C4D67BB9061365B8023FF14...,GalleryClicked,OTHER,1706799385679,RENTAL,2024-02-01 14:56:25.679,0.008128,...,50.0,50.0,1,1,0.0,0.0,"['AIR_CONDITIONING', 'KITCHEN', 'ELEVATOR', 'I...",1965817,283830,dt=2024-02-01
4,5960DD7B6E63AAA7CA77A30DF9E45C5652F1E4AE17A3B4...,B7DF7E93D4FF82B6B573B0A9ACED39EF9546D8A6789A05...,5960DD7B6E63AAA7CA77A30DF9E45C5652F1E4AE17A3B4...,1D7A1C175FA8FBF963E9915C4D67BB9061365B8023FF14...,ListingRendered,VISIT,1706799952847,RENTAL,2024-02-01 15:05:52.847,0.010837,...,50.0,50.0,1,1,0.0,0.0,"['AIR_CONDITIONING', 'KITCHEN', 'ELEVATOR', 'I...",1965817,283830,dt=2024-02-01


In [6]:
data.columns

Index(['unified_user_id', 'listing_id', 'user_id', 'anonymous_id', 'name_raw',
       'event_type', 'collector_timestamp', 'business_type', 'event_ts',
       'rating', 'price', 'state', 'city', 'neighborhood', 'usable_areas',
       'total_areas', 'bathrooms', 'bedrooms', 'suites', 'parking_spaces',
       'amenities', 'listing_id_numeric', 'user_numeric_id', 'dt'],
      dtype='object')

In [7]:
def standardize_column_names(df):
    cols_to_delete = [
        'event_ts', 'unified_user_id', 'listing_id', 'user_id', 'anonymous_id', 'dt'
    ]

    rename_mapping = {
        'user_numeric_id': 'user',
        'listing_id_numeric': 'item',
        'rating': 'label',
    }


    return df.drop(columns=cols_to_delete).rename(columns=rename_mapping)

In [8]:
data = standardize_column_names(data)
data

,name_raw,event_type,collector_timestamp,business_type,label,price,state,city,neighborhood,usable_areas,total_areas,bathrooms,bedrooms,suites,parking_spaces,amenities,item,user
0,GalleryClicked,OTHER,1706797334226,RENTAL,0.008128,1150000.0,Espírito Santo,Vila Velha,Praia da Costa,107.0,117.0,3,3,1.0,2.0,"['GYM', 'BARBECUE_GRILL', 'KITCHEN', 'ELEVATOR...",1860761,329768
1,GalleryClicked,OTHER,1706799383732,RENTAL,0.008128,175000.0,Espírito Santo,Vila Velha,Itapuã,50.0,50.0,1,1,0.0,0.0,"['AIR_CONDITIONING', 'KITCHEN', 'ELEVATOR', 'I...",1965817,283830
2,ListingRendered,VISIT,1706754084854,RENTAL,0.010797,1650000.0,Espírito Santo,Vila Velha,Itapuã,260.0,260.0,4,4,2.0,2.0,[],1614965,952233
3,GalleryClicked,OTHER,1706799385679,RENTAL,0.008128,175000.0,Espírito Santo,Vila Velha,Itapuã,50.0,50.0,1,1,0.0,0.0,"['AIR_CONDITIONING', 'KITCHEN', 'ELEVATOR', 'I...",1965817,283830
4,ListingRendered,VISIT,1706799952847,RENTAL,0.010837,175000.0,Espírito Santo,Vila Velha,Itapuã,50.0,50.0,1,1,0.0,0.0,"['AIR_CONDITIONING', 'KITCHEN', 'ELEVATOR', 'I...",1965817,283830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118032,GalleryClicked,OTHER,1722523405379,RENTAL,0.015948,170000.0,Espírito Santo,Vila Velha,Itapuã,35.0,35.0,1,1,0.0,0.0,"['ELEVATOR', 'INTERCOM', 'BALCONY', 'SECURITY_...",1011353,830147
118033,GalleryClicked,OTHER,1722544919741,RENTAL,0.015948,290000.0,Espírito Santo,Vila Velha,Coqueiral de Itaparica,52.0,52.0,1,2,0.0,1.0,[],1714905,176967
118034,GalleryClicked,OTHER,1722500780608,RENTAL,0.015948,1150000.0,Espírito Santo,Vila Velha,Praia da Costa,107.0,107.0,3,3,1.0,2.0,"['BARBECUE_GRILL', 'ELEVATOR', 'POOL', 'SAUNA']",1611344,96812
118035,GalleryClicked,OTHER,1722514991856,RENTAL,0.015948,135000.0,Espírito Santo,Vila Velha,Nova Itaparica,25.0,25.0,1,1,0.0,0.0,['PETS_ALLOWED'],2310718,971483


In [9]:
data.columns

Index(['name_raw', 'event_type', 'collector_timestamp', 'business_type',
       'label', 'price', 'state', 'city', 'neighborhood', 'usable_areas',
       'total_areas', 'bathrooms', 'bedrooms', 'suites', 'parking_spaces',
       'amenities', 'item', 'user'],
      dtype='object')

In [10]:
data['dt_datetime'] = pd.to_datetime(data['collector_timestamp'], unit='ms')

# converte para epoch em segundos
data['time'] = data['dt_datetime'].astype('int64') // 10**9

In [11]:
data = data.drop(columns=['collector_timestamp', 'dt_datetime'])
data

,name_raw,event_type,business_type,label,price,state,city,neighborhood,usable_areas,total_areas,bathrooms,bedrooms,suites,parking_spaces,amenities,item,user,time
0,GalleryClicked,OTHER,RENTAL,0.008128,1150000.0,Espírito Santo,Vila Velha,Praia da Costa,107.0,117.0,3,3,1.0,2.0,"['GYM', 'BARBECUE_GRILL', 'KITCHEN', 'ELEVATOR...",1860761,329768,1706797334
1,GalleryClicked,OTHER,RENTAL,0.008128,175000.0,Espírito Santo,Vila Velha,Itapuã,50.0,50.0,1,1,0.0,0.0,"['AIR_CONDITIONING', 'KITCHEN', 'ELEVATOR', 'I...",1965817,283830,1706799383
2,ListingRendered,VISIT,RENTAL,0.010797,1650000.0,Espírito Santo,Vila Velha,Itapuã,260.0,260.0,4,4,2.0,2.0,[],1614965,952233,1706754084
3,GalleryClicked,OTHER,RENTAL,0.008128,175000.0,Espírito Santo,Vila Velha,Itapuã,50.0,50.0,1,1,0.0,0.0,"['AIR_CONDITIONING', 'KITCHEN', 'ELEVATOR', 'I...",1965817,283830,1706799385
4,ListingRendered,VISIT,RENTAL,0.010837,175000.0,Espírito Santo,Vila Velha,Itapuã,50.0,50.0,1,1,0.0,0.0,"['AIR_CONDITIONING', 'KITCHEN', 'ELEVATOR', 'I...",1965817,283830,1706799952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118032,GalleryClicked,OTHER,RENTAL,0.015948,170000.0,Espírito Santo,Vila Velha,Itapuã,35.0,35.0,1,1,0.0,0.0,"['ELEVATOR', 'INTERCOM', 'BALCONY', 'SECURITY_...",1011353,830147,1722523405
118033,GalleryClicked,OTHER,RENTAL,0.015948,290000.0,Espírito Santo,Vila Velha,Coqueiral de Itaparica,52.0,52.0,1,2,0.0,1.0,[],1714905,176967,1722544919
118034,GalleryClicked,OTHER,RENTAL,0.015948,1150000.0,Espírito Santo,Vila Velha,Praia da Costa,107.0,107.0,3,3,1.0,2.0,"['BARBECUE_GRILL', 'ELEVATOR', 'POOL', 'SAUNA']",1611344,96812,1722500780
118035,GalleryClicked,OTHER,RENTAL,0.015948,135000.0,Espírito Santo,Vila Velha,Nova Itaparica,25.0,25.0,1,1,0.0,0.0,['PETS_ALLOWED'],2310718,971483,1722514991


In [12]:
order_columns = [
    'user', 'item','label', 'time', 'name_raw', 'event_type','business_type',
    'price', 'state', 'city', 'neighborhood', 'usable_areas', 'total_areas', 'bathrooms', 'bedrooms', 'suites', 'parking_spaces', 'amenities'
]


# Aplique a nova ordem
data = data[order_columns]

In [13]:
from libreco.data import DatasetFeat

from libreco.algorithms import DeepFM
from libreco.data import DatasetFeat, split_by_ratio_chrono

train_data, eval_data = split_by_ratio_chrono(data, test_size=0.2)

sparse_col = ['state', 'city', 'neighborhood', 'amenities']
dense_col = [
    'price', 'usable_areas', 'bathrooms', 'bedrooms',
    'suites', 'parking_spaces',
]
user_col = []
item_col = sparse_col + dense_col

for col in sparse_col:
    train_data[col] = train_data[col].fillna('desconhecido')
    eval_data[col] = eval_data[col].fillna('desconhecido')
for col in dense_col:
    median_val = train_data[col].median()
    train_data[col] = train_data[col].fillna(median_val)
    eval_data[col] = eval_data[col].fillna(median_val)

train_data, data_info = DatasetFeat.build_trainset(
    train_data=train_data,
    user_col=user_col,
    item_col=item_col,
    sparse_col=sparse_col,
    dense_col=dense_col,
    multi_sparse_col=[],
    pad_val=["missing"],  # specify padding value
)

eval_data = DatasetFeat.build_evalset(eval_data)

print("Dataset construído com sucesso, valores nulos foram tratados!")

Instructions for updating:
non-resource variables are not supported in the long term
Dataset construído com sucesso, valores nulos foram tratados!


In [14]:
print(data_info)

n_users: 18056, n_items: 300, data density: 1.8010 %


In [ ]:

deepfm = DeepFM(
    "ranking",
    data_info,
    embed_size=32,
    n_epochs=250,
    lr=1e-3,
    lr_decay=False,
    reg=None,
    batch_size=2048,
    num_neg=1,
    use_bn=False,
    dropout_rate=None,
    hidden_units=(1024, 512, 256),
    tf_sess_config=None,
    multi_sparse_combiner="sqrtn",  # specify multi_sparse combiner
)

deepfm.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    shuffle=True,
    eval_data=eval_data,
    metrics=[
        "loss",
        "balanced_accuracy",
        "roc_auc",
        "pr_auc",
        "precision",
        "recall",
        "map",
        "ndcg",
    ],
)


2025-09-30 22:01:00.907423: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-09-30 22:01:00.909990: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-09-30 22:01:00.910252: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Training start time: 2025-09-30 22:01:01
total params: 1,665,659 | embedding params: 616,782 | network params: 1,048,877


2025-09-30 22:01:01.770855: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
train:   0%|          | 0/96 [00:00<?, ?it/s]2025-09-30 22:01:02.443605: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:225] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2025-09-30 22:01:02.443627: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:228] Used ptxas at ptxas
2025-09-30 22:01:02.443678: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-09-30 22:01:02.561670: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-09-30 22:01:02.612744: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile ge

Epoch 1 elapsed: 1.329s
	 train_loss: 598678.4375


eval_listwise: 100%|██████████| 326/326 [00:01<00:00, 184.25it/s]


...

train: 100%|██████████| 96/96 [00:00<00:00, 177.19it/s]


Epoch 3 elapsed: 0.545s
	 train_loss: 10036.9805


eval_listwise: 100%|██████████| 326/326 [00:01<00:00, 182.39it/s]


	 eval log_loss: 11.3252
	 eval balanced_accuracy: 0.6850
	 eval roc_auc: 0.6871
	 eval pr_auc: 0.7623
	 eval precision@10: 0.0023
	 eval recall@10: 0.0128
	 eval map@10: 0.0060
	 eval ndcg@10: 0.0097


train:  58%|█████▊    | 56/96 [00:00<00:00, 183.27it/s]

In [15]:
print("prediction: ", deepfm.predict(user=1, item=2333))
print("recommendation: ", deepfm.recommend_user(user=1, n_rec=7))

Detect 1 unknown interaction(s), position: [0]
prediction:  0.0
Detect unknown user: 1
recommendation:  {1: array([1741828,  616624,  816756, 2015176, 1611344, 2167208,   51820],
      dtype=int32)}
